# Lab Assignment 4: Using APIs in Python
## DS 6001: Practice and Application of Data Science

### Instructions
Please answer the following questions as completely as possible using text, code, and the results of code as needed. Format your answers in a Jupyter notebook. To receive full credit, make sure you address every part of the problem, and make sure your document is formatted in a clean and professional way.

In this lab, you will work with the public API provided by [genius.com](https://genius.com/), a website that calls itself "the world’s biggest collection of song lyrics and musical knowledge." You will need to read the API documentation carefully, acquire an access key, and use it without sharing it to pull data from this API into Python. You will also practice using a library made specifically to wrap around `requests` to make calling from the Genius API easier.

## Problem 0
Import the following libraries:

In [2]:
import numpy as np
import pandas as pd
import requests
import json
import os
import dotenv
import sys
sys.tracebacklimit = 0 # turn off the error tracebacks

## Problem 1
The Genius API documentation is here: https://docs.genius.com/#/getting-started-h1. Read through the documentation carefully. Although the Genius API is free and public, it still requires users an access key to use the API. In this case, Genius provides users with three codes: a client ID, a client ID secret, and a client access token. Use the documentation to find a way to obtain these codes for yourself. Write a paragraph that describes all of the steps you needed to take (but DO NOT list your access codes in this paragraph).

Some hints and cautions: 

1. Before you can use the API, you will need a regular, free user account with Genius. Sign-up here: https://genius.com/signup_or_login

2. Genius's API is built to support third-party app development, not data scientists. The language is entirely geared toward app development. Under "Authentication" there are instructions to third-party developers for guiding their own users in getting access keys. That's not relevant to getting access for yourself. It's not hard to get an access key, but the guidance here is not very clear. Be patient and read everything in the Authentication section carefully.

3. When you arrive at the page that allows you to register for API access keys, the language is still geared toward app development. You will be prompted to name your app and provide the URLs associated with the app. It doesn't much matter what you name your app, and I just used the Collab main page (https://collab.its.virginia.edu/portal?containerLogin=true) for the URLs.

4. When you get your codes, copy them in a text file. In problem 2 you will copy these codes over again to a `.env` file. [4 points]

* First I went to the provided docs link, and under _Registering Your Application_ I learned that I need to go the _API Client management page_ in order to obtain my `client_id` and `client_secret`. On [this](https://genius.com/api-clients) page I clicked the _New API Client_, it's hard to miss. I then provided an app name and for the URLs, I provided the collab main page, then clicked save. After this, I was provided with the necessary keys for this API. Finally, I saved these keys in my `.env` file.

## Problem 2
Create a `.env` file for this project. Open it, copy your access codes into it, and save it. Then use Python code to load the environmental variables contained in the `.env` file, and create variables that contain each of the three codes. (You can print these variables to make sure it worked, but do not allow your access keys to display in your notebook). [4 points]

In [3]:
dotenv.load_dotenv()
clientid = os.getenv('genius')
clientsecret = os.getenv('geniussecret')

## Problem 3
The root for all Genius APIs is https://api.genius.com. Find the endpoint for the Search API. (You will have to click the "Authorize with Genius" button in the upper-right corner if you haven't already done so). Use the `requests` library to issue a search for Bob Dylan. Genius's API is organized in a way that every individual artist has his or her own API endpoint. Display a portion of the JSON output that displays the API endpoint path for the data on Bob Dylan. 

Hint: to authenticate, specify your access token (not your client ID or client secret) as the `access_token` parameter. You will have to dig around the JSON output to find the artist ID, but it is listed under `primary_artist` several branches down the JSON tree. [4 points]

In [24]:
url = 'https://httpbin.org/user-agent'
r = requests.get(url)
mjson = r.json()
useragent = mjson['user-agent']

In [25]:
root = 'https://api.genius.com/'
endpoint = 'search?q=Bob%20Dylan'

dotenv.load_dotenv()

tok = 'Bearer '+os.getenv('geniustok')

headers = {'Authorization':tok,
           'User-Agent':useragent}

r = requests.get(root+endpoint,headers=headers)

In [26]:
gjson = json.loads(r.text)
gjson['response']['hits'][0]['result']['primary_artist']

{'api_path': '/artists/181',
 'header_image_url': 'https://images.genius.com/e87fb11dd7f33cc7fd1a8915ad36f7c9.1000x667x1.png',
 'id': 181,
 'image_url': 'https://images.genius.com/a94817af6c91a49c07fad51829d30f4e.640x640x1.jpg',
 'is_meme_verified': False,
 'is_verified': False,
 'name': 'Bob Dylan',
 'url': 'https://genius.com/artists/Bob-dylan'}

In [28]:
artistendpoint = gjson['response']['hits'][0]['result']['primary_artist']['api_path']

## Problem 4
Add `/songs` to the end of the the endpoint path you found in problem 3 and use this path to request the 20 most popular Bob Dylan songs. Organize these data in a `pandas` data frame. [4 points]

In [29]:
headers = {'Authorization':tok,
           'User-Agent':useragent}

params = {'sort':'popularity'}

newend = artistendpoint+'/songs'
r = requests.get(root+newend,headers=headers,params=params)
bobjson = json.loads(r.text)

In [30]:
bobdf = pd.json_normalize(bobjson,record_path=['response','songs'])
bobdf

,annotation_count,api_path,artist_names,full_title,header_image_thumbnail_url,header_image_url,id,lyrics_owner_id,lyrics_state,path,...,stats.hot,stats.pageviews,primary_artist.api_path,primary_artist.header_image_url,primary_artist.id,primary_artist.image_url,primary_artist.is_meme_verified,primary_artist.is_verified,primary_artist.name,primary_artist.url
0,15,/songs/96286,USA For Africa,We Are the World by USA For Africa,https://images.genius.com/d328397b28953bd84465...,https://images.genius.com/d328397b28953bd84465...,96286,4733728,complete,/Usa-for-africa-we-are-the-world-lyrics,...,False,699630,/artists/370890,https://images.genius.com/3fb2d9f68c911b547339...,370890,https://images.genius.com/3fb2d9f68c911b547339...,False,False,USA For Africa,https://genius.com/artists/Usa-for-africa
1,12,/songs/79424,Bob Dylan,Blowin' in the Wind by Bob Dylan,https://images.genius.com/84e1705bc64495197216...,https://images.genius.com/84e1705bc64495197216...,79424,73267,complete,/Bob-dylan-blowin-in-the-wind-lyrics,...,False,652032,/artists/181,https://images.genius.com/e87fb11dd7f33cc7fd1a...,181,https://images.genius.com/a94817af6c91a49c07fa...,False,False,Bob Dylan,https://genius.com/artists/Bob-dylan
2,1,/songs/68146,Adele,Make You Feel My Love by Adele,https://images.genius.com/f248cc85ccaab603ce88...,https://images.genius.com/f248cc85ccaab603ce88...,68146,82481,complete,/Adele-make-you-feel-my-love-lyrics,...,False,642925,/artists/2300,https://images.genius.com/87aa5d8c32965a10e0e7...,2300,https://images.genius.com/8a23ab928ccfce13accf...,False,False,Adele,https://genius.com/artists/Adele
3,19,/songs/62605,Bob Dylan,The Times They Are A-Changin' by Bob Dylan,https://images.genius.com/3f8af2a0b75202890de6...,https://images.genius.com/3f8af2a0b75202890de6...,62605,49202,complete,/Bob-dylan-the-times-they-are-a-changin-lyrics,...,False,558655,/artists/181,https://images.genius.com/e87fb11dd7f33cc7fd1a...,181,https://images.genius.com/a94817af6c91a49c07fa...,False,False,Bob Dylan,https://genius.com/artists/Bob-dylan
4,31,/songs/54784,Bob Dylan,Like a Rolling Stone by Bob Dylan,https://images.genius.com/a31244adf46435e0a958...,https://images.genius.com/a31244adf46435e0a958...,54784,7,complete,/Bob-dylan-like-a-rolling-stone-lyrics,...,False,526260,/artists/181,https://images.genius.com/e87fb11dd7f33cc7fd1a...,181,https://images.genius.com/a94817af6c91a49c07fa...,False,False,Bob Dylan,https://genius.com/artists/Bob-dylan
5,7,/songs/103982,Guns N' Roses,Knockin’ on Heaven’s Door by Guns N' Roses,https://images.genius.com/cb2614550098bf753ab3...,https://images.genius.com/cb2614550098bf753ab3...,103982,168784,complete,/Guns-n-roses-knockin-on-heavens-door-lyrics,...,False,524718,/artists/637,https://images.genius.com/9127ece8b46f708fcc16...,637,https://images.genius.com/d8c0421f5938f6666ed4...,False,False,Guns N’ Roses,https://genius.com/artists/Guns-n-roses
6,15,/songs/98664,Bob Dylan,All Along the Watchtower by Bob Dylan,https://images.genius.com/40169c4a86e479f32e7a...,https://images.genius.com/40169c4a86e479f32e7a...,98664,779,complete,/Bob-dylan-all-along-the-watchtower-lyrics,...,False,521964,/artists/181,https://images.genius.com/e87fb11dd7f33cc7fd1a...,181,https://images.genius.com/a94817af6c91a49c07fa...,False,False,Bob Dylan,https://genius.com/artists/Bob-dylan
7,128,/songs/5393247,Bob Dylan,Murder Most Foul by Bob Dylan,https://images.genius.com/7fbaf838ffdf99315408...,https://images.genius.com/7fbaf838ffdf99315408...,5393247,3360167,complete,/Bob-dylan-murder-most-foul-lyrics,...,False,513064,/artists/181,https://images.genius.com/e87fb11dd7f33cc7fd1a...,181,https://images.genius.com/a94817af6c91a49c07fa...,False,False,Bob Dylan,https://genius.com/artists/Bob-dylan
8,10,/songs/84846,Bob Dylan,"Don't Think Twice, It's All Right by Bob Dylan",https://images.genius.com/84e1705bc64495197216...,https://images.genius.com/84e1705bc64495197216...,84846,125524,complete,/Bob-dylan-dont-think-twice-its-all-right-lyrics,...,False,454673,/arti

## Problem 5
Install and import the `lyricsgenius` library in Python, which is a wrapper around `requests` that works specifically with the Genius API. . Follow the guide on the GitHub repository for this library (https://github.com/johnwmillr/LyricsGenius) for instructions on using the library. Use the `lyricsgenius` library to download and display the lyrics to "Tangled Up in Blue" by Bob Dylan. [4 points]

In [36]:
import lyricsgenius as lg

genius = lg.Genius(os.getenv('geniustok'))
song = genius.search_song('Tangled Up in Blue','Bob Dylan')
print(song.lyrics)

Searching for "Tangled Up in Blue" by Bob Dylan...
Done.
103 ContributorsTangled Up in Blue Lyrics[Verse 1]
Early one morning the sun was shining
I was laying in bed
Wondering if she'd changed at all
If her hair was still red
Her folks they said our lives together
Sure was going to be rough
They never did like Mama's homemade dress
Papa's bankbook wasn't big enough
And I was standing on the side of the road
Rain falling on my shoes
Heading out for the East Coast
Lord knows I've paid some dues
Getting through
Tangled up in blue

[Verse 2]
She was married when we first met
Soon to be divorced
I helped her out of a jam, I guess
But I used a little too much force
We drove that car as far as we could
Abandoned it out west
Split up on a dark sad night
Both agreeing it was best
She turned around to look at me
As I was walking away
I heard her say over my shoulder
"We'll meet again someday
On the avenue"
Tangled up in blue
See Bob Dylan LiveGet tickets as low as $41You might also like[Verse 3]